# Global terrorist database (GTD): predicting perpetrator groups

## Problem description
"Use attack type, weapons used, description of the attack, etc. to build a model that can predict what group may have been responsible for an incident."

## What is the problem?
### Informal description
We have data about terrorist attacks but we do now know who commited many of them. Our model should tell us who might be behind a new incident.

### Formal description
* Experience: Data about previous terrorist attacks.
* Task: Predict which terrorist group might have been responsible for a new terrorist attacks.
* Performance (baseline): The ratio of correctly predicted terrorist groups.

### Assumptions
* There are clearly defined groups behind the indicent.
* Terrorist groups have a robust and consistent method which does not change significantly.
* Terrorists approach "terror problems" similarly across groups, regions and time periods and therefore the patterns of some terrorist acts can help us to explain others or predict future ones.
* Particular definition of terrorism (see appendixes)
* The information sources and the information itself are valid

### Similar problems
Domain similarity: Other violent but non-terrorist acts

## Why does the problem needs to be solved?
### Motivation
* Violent acts, like terrorism are wrong and should be stopped.
* Better understanding terrorism might help us to also understand the motivation of the perpetrators and the broader circumstances causing them.
* Ideally, we would be able to identify terrorist groups--even if they do not have any incident history in the database yet--which are planning an attact, their location, time and other specifics. We cannot do this based only on this dataset

### Possible benefits
* Having a better picture of the working of terrorist groups
* Identify common patterns among terrorist acts and their perpetrators

### Solution use
* We can estimate better unknown past and future perpetrators.
* Predict and prevent future events from happening.
* Identify organizing principles behind terrorist acts and groups.

## How would I solve the problem without machine learning?
* Ethnographic approach: Interviewing perpetartors and their peers to gain knowledge about the story of their terrorist projects. What were their motivations, what were their aims, what circumstances did they have to follow, what practicalities did they need to attend?
* Macrosociological approach: Examining the actual socio-economic patterns within the region preceding the terrorist acts and the material-technological circumstances with which the perpetrators needed to work with.

# The Data
The project examines the following dataset:
> National Consortium for the Study of Terrorism and Responses to Terrorism (START). (2016). Global Terrorism Database [Data file]. Retrieved from https://www.start.umd.edu/gtd

Data is missing for 1993.
```python
gtd.iyear.value_counts().sort_index()
```
Nonetheless, these are also available in a separate file.

## GTD history timeline
* 2001 University of Mariland gains data from Pinkerton about terrorism events from 1970 to 1997 (GTD1)
* 2005 Digitization: corrections and adding information
* April 2006 Funding to extend the data beyond 1997 through 2007 from archival sources and with a different concept of terrorism (GTD2)
* 2008 data collection is finished, applying the new inclusion criteria also on the previous data
* Spring of 2008- Spring of 2012: ISVG collects data betwen April 2008 - October 2011. This is integrated and the existing data is improved.
* 2012 - Data starting with November 2011 is conducted by START,
    - "As a result of these improvements, **a direct comparison between 2011 and 2012 data likely overstates the increase in total attacks and fatalities** worldwide during this time period."
    
in 2014, [other researchers raised issues](https://www.washingtonpost.com/news/monkey-cage/wp/2014/08/11/how-to-fix-the-flaws-in-the-global-terrorism-database-and-why-it-matters/?noredirect=on) about the GTD's failure to make explicit its temporal differences. As of the time of access, the dataset description contains the following warning:

> Users should note that differences in levels of attacks before and after January 1, 1998, before and after April 1, 2008, and before and after January 1, 2012 may be at least partially explained by differences in data collection

## Further specs
- Cases when the **"coder noted some uncertainty whether an incident meets all of the criteria for inclusion ("Doubt Terrorism Proper,")"**
- The GTD includes failed attacs, but does not include foiled or failed plots and violent threats where no action were taken.
- No state terrorims is included.
- Sources range from well-known international news agencies to English translations of local papers written in numerous foreign languages.
- Prior to 2012, identifying a year’s worth of terrorist incidents for inclusion in the GTD typically involved the use of approximately 300 unique news sources. By comparison, the 2012 update is based on a pool of over 1500 unique news sources.

## Criteria for terrorism
Three main criteria of terrorism
- Intentional
- Entails violence (against property/people)
- Sub-national actor

In addition, at least two of the following:
- Provided some social goal (i.e. not only for profit)
- Intention to convey message to the broader public
- Outside humanitarian law

## Similar works

There are a number of publicly available papers describing machine learning projects with the GTD. A few of them aims to predict perpetrator groups, most of them exclusively focusing on incidents in India. They are somehow built upon each other, so I reviewed closely the two most recent ones. One paper used 'Factor Analysis of Mixed Data' to select attributes for the model, imputed missing values, and used the data between 1990 and 2014 to predict perpetrators of the 2015 incidents. It reported a 73.2% accuracy with SVM <a id='1'> [[1]](#f1) </a>. Another one used data from 1970 and 2015 and--after some feature engineering, cleaning and rebalancing--it used C4.5 (or J48 in WEKA) to correctly classify 98.7936% of the instances with 0.988 F-measure and 0.998 ROC AUC <a id='2'> [[2]](#f2) </a>.

## Importing libraries

## Fundamentals

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot

## EDA and preprocessing

In [2]:
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.ensemble import IsolationForest

/home/andras/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
from imblearn.over_sampling import SMOTE

## Cross validation

In [4]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate, StratifiedKFold, StratifiedShuffleSplit

## Models

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

## Metrics

In [6]:
from sklearn.metrics import accuracy_score, precision_score

## Memory management
### Setting the temp folder
This is required for "jobs=-1" to work on Kaggle at some cases (see https://www.kaggle.com/getting-started/45288#292143)

In [7]:
%env JOBLIB_TEMP_FOLDER=/tmp

env: JOBLIB_TEMP_FOLDER=/tmp


In [8]:
import gc

# Loading the data

In [9]:
# Instead of the excel from their homepage, I use the csv version they uploaded to Kaggle
#gtd = pd.read_excel("globalterrorismdb_0617dist.xlsx")
gtd = pd.read_csv("../input/globalterrorismdb_0617dist.csv", encoding='ISO-8859-1')
gtd = gtd.sample(frac=0.1, random_state=4721)

/home/andras/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,6,31,33,53,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# <a id='f1'>[[1]](#1)</a> Terrorism Analytics: Learning to Predict the Perpetrator

D. Talreja, J. Nagaraj, N. J. Varsha and K. Mahesh, "Terrorism analytics: Learning to predict the perpetrator," 2017 International Conference on Advances in Computing, Communications and Informatics (ICACCI), Udupi, 2017, pp. 1723-1726. doi: 10.1109/ICACCI.2017.8126092

Because the authors trained their model on the data between 1990 and 2014 and tested it on the 2015 data, I will do the same here. Nonetheless, the exact number of examples I received here were not the same as what they reported which might be due to either update in the database since then or to differences in data cleaning (they do not provide code so I tried to reconstruct it based on their description).

In [84]:
ind1 = gtd[(gtd.iyear >= 1990) 
    & (gtd.iyear <= 2015) 
    & (gtd.country_txt == 'India') 
    & (gtd.gname != 'Unknown')] \
    .loc[:, ['iyear', 'attacktype1', 'targtype1', 'targsubtype1', 'weaptype1', 'latitude',
             'longitude', 'natlty1', 'property', 'INT_ANY', 'multiple', 'crit3', 'gname']]
ind1.shape

(550, 13)

In [85]:
# Here I do not impute missing values as they did and I also assume that they 
# dropped groups which are responsible only for single incident (within the examined period)
ind1 = ind1.dropna(how='any')

ind1.shape

(523, 13)

In [86]:
ind1 = ind1[ind1.gname.isin(ind1.gname.value_counts()[ind1.gname.value_counts() > 1].index.values)]
ind1.shape

(480, 13)

In [87]:
ind1_ = ind1.copy()

In [88]:
X = ind1_[ind1_.iyear < 2015].drop(columns='gname')
y = ind1_[ind1_.iyear < 2015].gname

In [89]:
validation_size = 0.2
seed = 17

In [90]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=validation_size, random_state=seed)
print(X_train.shape)
print(X_validation.shape)
print(y_train.shape)
print(y_validation.shape)


(352, 12)
(89, 12)
(352,)
(89,)


I try both SVM (the model they found to be the best) and Decision Tree Classifier (a model worked well in my analysis).

In [91]:
models = {"Decisiong Tree Classifier": DecisionTreeClassifier(),
          'Support Vector Classifier': SVC(gamma='auto'),
}

They reported only accuracy scores so I will do the same.

In [92]:
def predict_groups(models, X_train, y_train):
    for model in models:
        #print("\n{}:\n\n{}\n".format(model, models[model]))
          
        model_score = cross_val_score(models[model], X_train, y_train, cv=kfold, scoring='accuracy')
        print("\n{}:\n\tAccuracy: {} ({})".format(model, model_score.mean(), model_score.std()))

In [93]:
kfold = StratifiedKFold(n_splits=10, random_state=seed)
#kfold = KFold(n_splits=10, random_state=seed)

In the cross validation and without data imputations and feature extractio the Decision Tree Classifier produced a similar accuracy as they did (they reported 73.2%), but not with SVC.

In [94]:
predict_groups(models, X_train, y_train)

/home/andras/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/andras/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)



Decisiong Tree Classifier:
	Accuracy: 0.7622079318530932 (0.11764011751208127)

Support Vector Classifier:
	Accuracy: 0.5152506633151794 (0.1114993751737338)


Also testing the 2015 year data (which was their own method)

In [95]:
model = DecisionTreeClassifier()
model.fit(X, y)
xtest = ind1[ind1.iyear == 2015].drop(columns='gname')
ytest = ind1[ind1.iyear == 2015].gname
(model.predict(xtest)  == ytest).mean()

0.6410256410256411

In [96]:
model = SVC()
model.fit(X, y)
(model.predict(ind1[ind1.iyear == 2015].drop(columns='gname').dropna(how='any'))  == ind1[ind1.iyear == 2015].gname).mean()

0.3076923076923077

These are indeed much less than the reported results so I will try out their features with imputation.

# <a id='f2'>[[2]](#2)</a> An Efficient Modelling of Terrorist Groups in India using Machine Learning Algorithms

Varun Teja Gundabathula and V. Vaidhehi, An Efficient Modelling of Terrorist Groups in India using Machine Learning Algorithms, Indian Journal of Science and Technology, Vol 11(15), DOI: 10.17485/ijst/2018/v11i15/121766, April 2018

In [53]:
ind2 = gtd[(gtd.iyear >= 1970) 
    & (gtd.iyear <= 2015) 
    & (gtd.country_txt == 'India') 
    & (gtd.gname != 'Unknown')] \
    .loc[:, ['iyear', 'imonth', 'iday', 'extended', 'provstate', 'city', 'attacktype1_txt', 'targtype1_txt', 
             'nperps', 'weaptype1_txt', 'nkill', 'nwound', 'nhostkid', 'gname']]

ind2.shape

(629, 14)

Here, again, I receive a slightly different amount of the unique perpetrator groups from what the authors reported (270), probably due to update in the database.

In [54]:
ind2.gname.nunique()

98

The authors removed those groups which were linked to only one incident. 

In [55]:
ind2 = ind2[ind2.gname.isin(ind2.gname.value_counts()[ind2.gname.value_counts() > 1].index.values)]
ind2.shape

(579, 14)

In [56]:
ind2_ = ind2.copy()
ind2_.shape

(579, 14)

## Feature extraction

They created a new column from the existing ones.

In [57]:
ind2_.loc[:, ['nkill', 'nwound', 'nhostkid']].describe()

,nkill,nwound,nhostkid
count,571.000000,555.000000,75.00000
mean,2.073555,2.214414,3.76000
std,4.744509,7.644810,5.29875
min,0.000000,0.000000,1.00000
25%,0.000000,0.000000,1.00000
50%,1.000000,0.000000,2.00000
75%,2.000000,1.000000,4.00000
max,62.000000,100.000000,36.00000


They created a new column from the existing ones.

In [58]:
crits = {'nkill': (62, 124, 'abc'), 
         'nwound': (272, 544, 'def'), 
         'nhostkid': (400, 800, 'ghi')}

In [59]:
def naff(data, crits):
    n = pd.Series('_')
    for key, i in zip(crits.keys(), range(len(crits))):
        i = data.loc[:, key].copy()

        i[data.loc[:,key] == 0] = 'n'
        i[(data.loc[:,key] > 0) 
          & (data.loc[:,key] < crits[key][0])] = crits[key][2][2]
        i[(data.loc[:,key] <= crits[key][1]) 
          & (data.loc[:,key] >= crits[key][0])] = crits[key][2][1]
        i[data.loc[:,key] > crits[key][1]] = crits[key][2][0]

        n = pd.concat((n, i), axis=1) 

    return n.drop(columns=0)

In [60]:
naffect = naff(ind2_, crits)
naffect.head(10)

,nkill,nwound,nhostkid
0,NaN,NaN,NaN
9734,c,n,NaN
16307,c,f,NaN
19962,c,f,NaN
20851,c,n,NaN
20987,n,n,NaN
21028,c,n,NaN
21501,n,f,NaN
21580,c,n,NaN
21601,c,n,NaN


In [61]:
naffect.nhostkid.value_counts()

i    75
Name: nhostkid, dtype: int64

Dropping built-in missing codes

In [62]:
naffect.nhostkid[naffect.nhostkid == -99] = np.NaN
naffect.replace(np.NaN, 'n', inplace=True)

In [63]:
naffect = naffect.iloc[:,0] +  naffect.iloc[:,1] +  naffect.iloc[:,2]

The frequency of the new values

In [64]:
naffect.value_counts()

nnn    183
cnn    163
cfn    106
nfn     52
cni     37
nni     33
cfi      3
nfi      2
bfn      1
dtype: int64

We drop replace the old columns with the new one

In [65]:
ind2_.drop(columns=['nkill', 'nwound', 'nhostkid'], inplace=True)
ind2_['naffect'] = naffect

In [66]:
ind2_.nperps.where(ind2_.nperps != -99, 0, inplace=True)
ind2_.nperps.fillna(0, inplace=True)

In [67]:
ind2_.head()

,iyear,imonth,iday,extended,provstate,city,attacktype1_txt,targtype1_txt,nperps,weaptype1_txt,gname,naffect
117537,2013,5,30,0,Uttar Pradesh,Kosi Kalan,Bombing/Explosion,Private Citizens & Property,0.0,Explosives/Bombs/Dynamite,Vishwa Hindu Parishad (VHP),nfn
94010,2009,10,27,0,Jharkhand,Giridih district,Bombing/Explosion,Educational Institution,0.0,Explosives/Bombs/Dynamite,Communist Party of India - Maoist (CPI-Maoist),nnn
72495,2001,8,6,0,Maharashtra,Mumbai,Unarmed Assault,Religious Figures/Institutions,25.0,Melee,Vishwa Hindu Parishad (VHP),nfn
58342,1995,2,4,0,Jammu and Kashmir,Jammu district,Bombing/Explosion,Utilities,0.0,Explosives/Bombs/Dynamite,Kashmiri extremists,nnn
74240,2002,6,27,0,Jammu and Kashmir,Phraslan,Bombing/Explosion,Military,0.0,Explosives/Bombs/Dynamite,Hizbul Mujahideen (HM),cnn


## Rebalancing
For their best results they used the SMOTE overbalancing algorithm

In [68]:
ind2_.gname.value_counts()

Communist Party of India - Maoist (CPI-Maoist)              168
Sikh Extremists                                              78
Maoists                                                      68
United Liberation Front of Assam (ULFA)                      29
Lashkar-e-Taiba (LeT)                                        20
National Democratic Front of Bodoland (NDFB)                 18
Muslim Separatists                                           16
People's War Group (PWG)                                     14
Hizbul Mujahideen (HM)                                       13
Bodo Militants                                               12
Garo National Liberation Army                                11
Kashmiri extremists                                           9
National Liberation Front of Tripura (NLFT)                   9
Muslim Militants                                              9
People's Liberation Army (India)                              7
Jammu and Kashmir Liberation Front      

In [69]:
X = pd.get_dummies(ind2_.drop(columns='gname'), sparse=True)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(ind2_.gname)

print(X.shape)
print(y.shape)

(579, 484)
(579,)


In [70]:
dict(np.array(np.unique(y, return_counts=True)).T)

{0: 2,
 1: 2,
 2: 2,
 3: 2,
 4: 3,
 5: 2,
 6: 12,
 7: 168,
 8: 2,
 9: 5,
 10: 11,
 11: 2,
 12: 3,
 13: 13,
 14: 3,
 15: 5,
 16: 4,
 17: 9,
 18: 4,
 19: 2,
 20: 20,
 21: 4,
 22: 68,
 23: 3,
 24: 9,
 25: 16,
 26: 2,
 27: 3,
 28: 18,
 29: 9,
 30: 3,
 31: 3,
 32: 4,
 33: 2,
 34: 7,
 35: 5,
 36: 14,
 37: 5,
 38: 78,
 39: 2,
 40: 2,
 41: 2,
 42: 3,
 43: 29,
 44: 4,
 45: 2,
 46: 2,
 47: 4}

In [71]:
smote = SMOTE(ratio='all', k_neighbors=1, n_jobs=-1)

In [72]:
tXr, tyr = smote.fit_sample(X, y) 

In [73]:
print(X.shape)
print(tXr.shape)
print(y.shape)
print(tyr.shape)

(579, 484)
(8064, 484)
(579,)
(8064,)


Label frequencies before and after rebalancing

In [74]:
np.array([np.unique(y, return_counts=True)[0], np.unique(y, return_counts=True)[1]]).T

array([[  0,   2],
       [  1,   2],
       [  2,   2],
       [  3,   2],
       [  4,   3],
       [  5,   2],
       [  6,  12],
       [  7, 168],
       [  8,   2],
       [  9,   5],
       [ 10,  11],
       [ 11,   2],
       [ 12,   3],
       [ 13,  13],
       [ 14,   3],
       [ 15,   5],
       [ 16,   4],
       [ 17,   9],
       [ 18,   4],
       [ 19,   2],
       [ 20,  20],
       [ 21,   4],
       [ 22,  68],
       [ 23,   3],
       [ 24,   9],
       [ 25,  16],
       [ 26,   2],
       [ 27,   3],
       [ 28,  18],
       [ 29,   9],
       [ 30,   3],
       [ 31,   3],
       [ 32,   4],
       [ 33,   2],
       [ 34,   7],
       [ 35,   5],
       [ 36,  14],
       [ 37,   5],
       [ 38,  78],
       [ 39,   2],
       [ 40,   2],
       [ 41,   2],
       [ 42,   3],
       [ 43,  29],
       [ 44,   4],
       [ 45,   2],
       [ 46,   2],
       [ 47,   4]])

In [75]:
np.array([np.unique(tyr, return_counts=True)[0], np.unique(tyr, return_counts=True)[1]]).T

array([[  0, 168],
       [  1, 168],
       [  2, 168],
       [  3, 168],
       [  4, 168],
       [  5, 168],
       [  6, 168],
       [  7, 168],
       [  8, 168],
       [  9, 168],
       [ 10, 168],
       [ 11, 168],
       [ 12, 168],
       [ 13, 168],
       [ 14, 168],
       [ 15, 168],
       [ 16, 168],
       [ 17, 168],
       [ 18, 168],
       [ 19, 168],
       [ 20, 168],
       [ 21, 168],
       [ 22, 168],
       [ 23, 168],
       [ 24, 168],
       [ 25, 168],
       [ 26, 168],
       [ 27, 168],
       [ 28, 168],
       [ 29, 168],
       [ 30, 168],
       [ 31, 168],
       [ 32, 168],
       [ 33, 168],
       [ 34, 168],
       [ 35, 168],
       [ 36, 168],
       [ 37, 168],
       [ 38, 168],
       [ 39, 168],
       [ 40, 168],
       [ 41, 168],
       [ 42, 168],
       [ 43, 168],
       [ 44, 168],
       [ 45, 168],
       [ 46, 168],
       [ 47, 168]])

In [76]:
X_train, X_validation, y_train, y_validation = train_test_split(tXr, tyr, test_size=validation_size, random_state=seed)
print(X_train.shape)
print(X_validation.shape)
print(y_train.shape)
print(y_validation.shape)

(6451, 484)
(1613, 484)
(6451,)
(1613,)


In [77]:
del tXr
del tyr

gc.collect()

73

In [78]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

In [79]:
models = {"Decisiong Tree Classifier": DecisionTreeClassifier(),
          "K-Neighbors Classifier": KNeighborsClassifier(),
          "Gaussian Naive Bayes": GaussianNB(),
         }

In [80]:
def eval_models(models, X, y):
    """Evaluates selected model's prediction power on the cross-validated training datasets.
    Takes
        models: Dictionary of "model_name": model() pairs.
        X: predictor attributes
        y: target attribute
    """
    results = []
    for model in models:
        #print("Running {}...".format(model))
        #start = time.time()

        result = []
        result.append(model)

        model_score = cross_validate(models[model],
                                    X,
                                    y,
                                    scoring=['accuracy', # Evaluation metrics
                                             'precision_micro',
                                             'recall_micro',
                                             'f1_micro',
                                            # 'roc_auc'
                                            ],
                                    cv=kfold, # Cross-validation method
                                    n_jobs=-1,
                                    verbose=0,
                                    return_train_score=False)

        acc_mean = model_score['test_accuracy'].mean()
        acc_std = model_score['test_accuracy'].std()
        #auc_mean = model_score['test_roc_auc'].mean()
        #auc_std = model_score['test_roc_auc'].std()

        print("\n{}:\n\tAccuracy: {} ({})".format(model, acc_mean, acc_std)) #auc_std

        #print("\tROC AUC: {} ({})".format(auc_mean, auc_std))

        precision_micro_mean = model_score['test_precision_micro'].mean()
        precision_micro_std = model_score['test_precision_micro'].std()
        recall_micro_mean = model_score['test_recall_micro'].mean()
        recall_micro_std = model_score['test_recall_micro'].std()

        f1_micro_mean = model_score['test_f1_micro'].mean()
        f1_micro_std = model_score['test_f1_micro'].std()
        print("\tF1 micro: {} ({})".format(f1_micro_mean, f1_micro_std))

        #result = result + [acc_mean, acc_std, auc_mean, auc_std]

        dur = model_score['fit_time'].sum() + model_score['score_time'].sum()

        print("\tduration:{}\n".format(dur))
        #result.append(dur)

        #results.append(result)

The cross validation produces extremely high accuracies which was also reported by the paper. This is might be the result of overfitting and, therefore, would need more examination.

In [81]:
eval_models(models, X_train, y_train)


Decisiong Tree Classifier:
	Accuracy: 0.9780300755926051 (0.0054627297147576215)
	F1 micro: 0.9780300755926051 (0.0054627297147576215)
	duration:13.52476191520691


K-Neighbors Classifier:
	Accuracy: 0.9613700552864142 (0.0068195063114669914)
	F1 micro: 0.9613700552864142 (0.0068195063114669914)
	duration:23.754706144332886


Gaussian Naive Bayes:
	Accuracy: 0.9725687495957647 (0.003094838019634866)
	F1 micro: 0.9725687495957647 (0.003094838019634866)
	duration:21.11389708518982

